In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
import pandas as pd

%matplotlib inline
torch.cuda.empty_cache()

# Define directories, transformations, and labels


In [ ]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


labels=["Male","Smiling","Wavy_Hair"]
label_number=0


# Load CNN's

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3().to(device)

In [ ]:
model.load_state_dict(torch.load('12_9_5e.pt'))
model.eval()

# Feature Extractions

In [ ]:
# feature extraction for training
#check feature extrcation

# 162770
# 19867
# 19962

bathsize = 35
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
train_tested_samples=0
limit=100000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('Male_train.csv', mode='a', index=False, header=True) ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(data) 
    px.to_csv('Male_train.csv', mode='a', index=False, header=False)   ######################csv name 
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    train_tested_samples+=pred.size()[0]
    if train_tested_samples>limit:
        break

    print(correct)
    print(train_tested_samples)

#print precision
print(correct.item()/train_tested_samples)
print(train_tested_samples)

In [ ]:

#validation feature extraction 

# 162770
# 19867
# 19962

bathsize = 35
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
val_tested_samples=0
limit=19000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('male_val_img.csv', mode='a', index=False, header=True)  ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(data) 

    px.to_csv('male_val_img.csv', mode='a', index=False, header=False)  ##################change csv name  
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    val_tested_samples+=pred.size()[0]
    if val_tested_samples>limit:
        break

    print(correct)
    print(val_tested_samples)

#print precision
print(correct.item()/val_tested_samples)
print(val_tested_samples)

In [ ]:
#test feature extraction 

bathsize = 35
FE_ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Test")
FE_loader=DataLoader(FE_ds,batch_size=bathsize, shuffle=False, num_workers=workers)

device = 'cuda'

FE_data=iter(FE_loader)

#model=model.to(device)

model.eval()

correct=0
test_tested_samples=0
limit=19000


l = ['label']
range_int = range(1,2049)
r = [str(x) for x in range_int]
col = l + r   
px = pd.DataFrame(columns = col)  
px.to_csv('male_test_img.csv', mode='a', index=False, header=True)    ##################change csv name
    
    
for data in FE_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)
    
    output, extraction = model(sample)
    
#     print(output.shape)
#     print(extraction.shape)
#     print(extraction)
    
    #rediction, extraction = output
        
    target_val = target.cpu().detach().numpy()
    #label_val = np.reshape(target_val, (50,1))
    
#     print(target_val.shape)
    #print(label_val)    
    extract = extraction.cpu().detach().numpy()
    

    data = np.insert(extract, 0, target_val, axis = 1)
#     print(data.shape)
#     print(data)
    val_data = np.reshape(data, (bathsize, 2049))
 
    px = pd.DataFrame(val_data) 

    px.to_csv('male_test_img.csv', mode='a', index=False, header=False)   ##################change csv name
        
    
    pred=torch.argmax(output,dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    test_tested_samples+=pred.size()[0]
    if test_tested_samples>limit:
        break

    print(correct)
    print(test_tested_samples)


#print precision
print(correct.item()/test_tested_samples)
print(test_tested_samples)

In [ ]:
# this is just used to check the output 

df_attr_train=pd.read_csv(labels_dir, nrows = train_tested_samples, usecols=["image_id",labels[label_number]]) 
# df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = val_tested_samples, usecols=["image_id",labels[label_number]])
# df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = test_tested_samples, usecols=["image_id",labels[label_number]])

# #change -1 to 0 in csv, remove this if not needed
df_attr_train.replace(-1,0,inplace=True)
#df_attr_val.replace(-1,0,inplace=True)
# df_attr_test.replace(-1,0,inplace=True)



In [ ]:
df_attr_train.info()

In [ ]:
df_attr_train.head(n=37)

In [ ]:
df_attr_train.tail(n=15)

In [ ]:
df_attr_train=pd.read_csv(labels_dir, nrows = 100030, usecols=["image_id"]) 
train_imgs = df_attr_train.to_numpy()
traindata = pd.read_csv('smile_train_img.csv')
traindata.insert(0, 'img_id', train_imgs)
traindata.to_csv('smile_train.csv', mode='a', index=False)

# df_attr_val=pd.read_csv(labels_dir, skiprows = [i for i in range(1,162771)], nrows = val_tested_samples, usecols=["image_id"])
# val_imgs = df_attr_val.to_numpy()
# valdata = pd.read_csv('smile_val_img.csv')
# #valdata.insert(0, 'label_from_csv', val_imgs[:,1])
# valdata.insert(0, 'img_id', val_imgs[:,0])
# valdata.to_csv('smile_val.csv', mode='a', index=False)


# df_attr_test=pd.read_csv(labels_dir, skiprows = [i for i in range(1,182638)], nrows = test_tested_samples, usecols=["image_id"])
# test_imgs = df_attr_test.to_numpy()
# testdata = pd.read_csv('smile_test_img.csv')
# testdata.insert(0, 'img name', test_imgs)
# testdata.to_csv('smile_test.csv', mode='a', index=False)

In [ ]:
traindata = pd.read_csv('smile_train_img.csv')

In [ ]:
traindata.info()